In [ ]:
%cd /ibex/user/slimhy/PADS/code/
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import json
import os
import torch
import trimesh
import zipfile

import util.misc as misc
import util.s2vs as s2vs

from datasets.CoMPaT.compat3D import SegmentedMeshLoader
from datasets.CoMPaT.utils3D.plot import label_to_RGB, FINE_RGB_RANGE
from datasets.metadata import (
    COMPAT_CLASSES,
    int_to_hex,
)
from util.misc import CUDAMesh, d_GPU, show_side_by_side
from voxelize.preprocess import mesh_to_manifold_plus

In [ ]:
IN_PATH = "/ibex/project/c2273/3DCoMPaT/manifold"
N_POINTS = 2*20
MAX_SAMPLE_SIZE = 2**17
NEAR_SURFACE_NOISE = 0.01
CONTAIN_METHOD = "occnet"

In [ ]:
def sample_batched(mesh, n_points, sampling_method="surface"):
    """
    Sampling points on the mesh in batches.
    """
    # Defining the sampling function
    sampling_fn = get_sampling_function(
        sampling_method, noise_std=NEAR_SURFACE_NOISE, contain_method=CONTAIN_METHOD
    )

    # Use batch sampling
    n_batches = n_points // MAX_SAMPLE_SIZE
    all_points, all_occs = [], []
    for k in range(n_batches):
        if k % 4 == 0:
            print("Sampling batch [%d/%d]" % (k + 1, n_batches))
        points, occs = sampling_fn(mesh, MAX_SAMPLE_SIZE)
        all_points += [points]
        all_occs += [occs]
        
    # Concatenate the results
    all_points = torch.cat(all_points, dim=0)
    all_occs = torch.cat(all_occs, dim=0)

    return all_points, all_occs

In [ ]:
# Get the list of all models
all_models = os.listdir(IN_PATH)
all_models = [os.path.basename(f).split(".")[0] for f in all_models]

# 
